In [1]:
print("hi")


hi


In [2]:
%pwd

'c:\\ProgramData\\ANURITI\\Medical-Chatbot-\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\ProgramData\\ANURITI\\Medical-Chatbot-'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
from langchain.document_loaders import PyPDFLoader

def load_pdf_file(data):
    loader = PyPDFLoader(data)  # Use PyPDFLoader directly for a single file
    documents = loader.load()
    return documents

In [7]:
extracted_data = load_pdf_file(data='C:/Users/ANURITI/Downloads/Medical_book.pdf')



In [8]:
#extracted_data

In [9]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [11]:
#text_chunks

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()

C:\Users\ANURITI\AppData\Local\Temp\ipykernel_15616\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [15]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [16]:
#query_result

In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [19]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

In [20]:

import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [21]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [22]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [23]:
docsearch

In [24]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [25]:
retrieved_docs = retriever.invoke("What is Acne?")

In [26]:
retrieved_docs

[Document(id='5b1bd578-bc43-4f53-b139-d8f31305ad21', metadata={'page': 39.0, 'page_label': '40', 'source': 'C:/Users/ANURITI/Downloads/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='6fbe343e-aa8f-44b1-ae4a-43dd90133983', metadata={'page': 39.0, 'page_label': '40', 'source': 'C:/Users/ANURITI/Downloads/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='a0afd77e-3d67-4d4a-900b-d6be964110b8', metadata={'page': 39.0, 'page_label': '40', 'source': 'C:/Users/ANURITI/Downloads/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26')]

In [27]:
import requests, os; os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")  # Assume the key is in your .env file
llm = lambda prompt: requests.post("https://api.groq.ai/v1/complete", json={"prompt": prompt, "max_tokens": 500, "temperature": 0.4}, headers={"Authorization": f"Bearer {os.environ['GROQ_API_KEY']}", "Content-Type": "application/json"}).json().get("text", "Error")


In [28]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import json


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
).format_messages(context="", input="What is Acne?")  # Convert to string format


In [29]:
import requests, os; os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")  # Assume the key is in your .env file
def llm(prompt):
    response = requests.post(
        "https://api.groq.ai/v1/complete",
        json={"prompt": prompt, "max_tokens": 500, "temperature": 0.4},
        headers={"Authorization": f"Bearer {os.environ['GROQ_API_KEY']}", "Content-Type": "application/json"}
    )
    return response.json().get("text", "Error")  # Ensures it returns a string


In [30]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
formatted_prompt = prompt.format_messages(context="", input="What is Acne?")
response = rag_chain.invoke({"input": formatted_prompt})


# Ensure the 'response' object is being correctly returned and is not None
if response is None:
    print("No response received!")
else:
    print("Response:", response)


TypeError: Object of type ChatPromptValue is not JSON serializable

In [75]:
import json
response_str = json.dumps(response, default=str)  # Will attempt to serialize non-serializables
print("Serialized Response:", response_str)


NameError: name 'response' is not defined

In [74]:
response = rag_chain.invoke({"input": "What is Acne?"})

# Check if the response is a ChatPromptValue and convert it to string
if isinstance(response, ChatPromptValue):  
    # Convert to string (this might depend on how the class is defined, use .content or .text if possible)
    response_str = str(response)  # or `response.content` if it has this attribute
else:
    response_str = str(response)  # Default conversion if it’s not a ChatPromptValue

print("Response (Stringified):", response_str)





TypeError: Object of type ChatPromptValue is not JSON serializable

In [71]:
print("Raw Response:", response)


NameError: name 'response' is not defined

In [61]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

TypeError: Object of type ChatPromptValue is not JSON serializable